In [19]:
import os
import pandas as pd
import sqlite3
from tqdm import tqdm

root_folder = 'C:/Users/20232075/Desktop/London Police Data'
db_path = 'crime_data.db'
batch_size = 2000

required_columns = [
    'Crime ID', 'Month', 'Reported by', 'Falls within',
    'Longitude', 'Latitude', 'Location',
    'LSOA code', 'Crime type', 'Last outcome category', 'Context'
]

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS crime (
        crimeID TEXT PRIMARY KEY,
        Month TEXT,
        Longitude REAL,
        Latitude REAL,
        LSOA_code TEXT,
        Type TEXT,
        Outcome TEXT
    )
''')
conn.commit()

insert_query = '''
    INSERT OR IGNORE INTO crime (
        crimeID, Month,Longitude, Latitude,
        LSOA_code, Type, Outcome
    ) VALUES (?, ?, ?, ?, ?, ?, ?);
'''

all_files = []
for subdir, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.csv'):
            all_files.append(os.path.join(subdir, file))

batch = []
file_count = 0
inserted_rows = 0

for file_path in tqdm(all_files, desc="Processing files", unit="file"):
    name_without_ext = file_path[:-4]
    if name_without_ext.lower().endswith('-street'):
        df = pd.read_csv(file_path)
        for col in required_columns:
            if col not in df.columns:
                df[col] = None

        df = df.rename(columns={
            'Crime ID': 'crimeID',
            'Month': 'Month',
            'Longitude': 'Longitude',
            'Latitude': 'Latitude',
            'LSOA code': 'LSOA_code',
            'Crime type': 'Type',
            'Last outcome category': 'Outcome',
        })

        df = df[['crimeID', 'Month', 'Longitude', 'Latitude',
                  'LSOA_code', 'Type', 'Outcome',]]

        records = list(df.itertuples(index=False, name=None))

        for record in tqdm(records, desc=f"Inserting {os.path.basename(file_path)}", leave=False):
            batch.append(record)
            if len(batch) >= batch_size:
                cursor.executemany(insert_query, batch)
                conn.commit()
                inserted_rows += len(batch)
                batch = []

        file_count += 1

if batch:
    cursor.executemany(insert_query, batch)
    conn.commit()
    inserted_rows += len(batch)

conn.close()
print(f"\nInserted {inserted_rows} rows from {file_count} files.")


Processing files: 100%|██████████| 72/72 [03:18<00:00,  2.76s/file]


Inserted 3386817 rows from 36 files.


In [20]:

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for file_path in tqdm(all_files, desc="Processing outcome files", unit="file"):
    name_without_ext = file_path[:-4]
    if name_without_ext.lower().endswith('-outcomes'):
        df = pd.read_csv(file_path)
        df = df[['Crime ID', 'Outcome type']].dropna(subset=['Crime ID'])

        update_records = list(df.itertuples(index=False, name=None))

        for crime_id, outcome in tqdm(update_records, desc=f"Updating {os.path.basename(file_path)}", leave=False):
            cursor.execute(
                "UPDATE crime SET Outcome = ? WHERE crimeID = ?;",
                (outcome, crime_id)
            )

conn.commit()
conn.close()
print("Outcome fields updated where applicable.")


Processing outcome files: 100%|██████████| 72/72 [01:12<00:00,  1.00s/file]


Outcome fields updated where applicable.


In [21]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) from crime"           )
result = cursor.fetchall()
print(result)

[(3309375,)]


In [22]:
import pandas as pd
import sqlite3
from tqdm import tqdm

tqdm.pandas()
db_path = 'crime_data.db'
ward_lsoa = "C:/Users/20232075/Downloads/LSOA_(2021)_to_Electoral_Ward_(2024)_to_LAD_(2024)_Best_Fit_Lookup_in_EW.csv"

mapping_df = pd.read_csv(ward_lsoa, usecols=["LSOA21CD", "WD24CD"])

conn = sqlite3.connect(db_path)

df = pd.read_sql_query("SELECT * FROM crime", conn)

merged_df = df.merge(mapping_df, how='left', left_on='LSOA_code', right_on='LSOA21CD')

merged_df.drop(columns=['LSOA21CD'], inplace=True)

merged_df.to_sql("crime", conn, if_exists="replace", index=False)

conn.close()


In [24]:
import sqlite3

conn = sqlite3.connect("crime_data.db")
cursor = conn.cursor()

# Step 1: Get all existing non-integer crimeIDs
cursor.execute("SELECT crimeID FROM crime WHERE crimeID IS NOT NULL")
existing_ids = {row[0] for row in cursor.fetchall()}

# Step 2: Get rows with NULL crimeID
cursor.execute("SELECT rowid FROM crime WHERE crimeID IS NULL")
null_rows = [row[0] for row in cursor.fetchall()]

# Step 3: Start assigning numeric IDs (e.g., 1, 2, 3...) that don't clash with existing ones
new_id = 1
for rowid in null_rows:
    # Skip if that number is already used as a string
    while str(new_id) in existing_ids:
        new_id += 1
    cursor.execute("UPDATE crime SET crimeID = ? WHERE rowid = ?", (str(new_id), rowid))
    new_id += 1

conn.commit()
conn.close()


In [17]:
cursor.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [18]:
os.remove("crime_data.db")

In [26]:
conn = sqlite3.connect("crime_data.db")
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT Type FROM crime")
crime_types = [row[0] for row in cursor.fetchall()]

print("Crime Types:")
for ctype in crime_types:
    print(ctype)

conn.close()


Crime Types:
Violence and sexual offences
Theft from the person
Anti-social behaviour
Burglary
Criminal damage and arson
Public order
Robbery
Vehicle crime
Drugs
Other theft
Shoplifting
Bicycle theft
Possession of weapons
Other crime


In [30]:
conn = sqlite3.connect("crime_data.db")

# Read directly into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM crime LIMIT 5", conn)

print(df)

conn.close()


                                             crimeID    Month  Longitude  \
0  6b1be5a8275fcda2982f6de7bb92e03499d33c7fa6316b...  2022-03  -0.445898   
1  b466ed98f60835f379377cda209f72a65f4b594d64b8a6...  2022-03  -0.530681   
2  cc667a5d8c2fa4f30ade960d334d4c1feac01170092470...  2022-03   0.876572   
3  a60d9d1cff047a67e9a22361a6aa27ad978ec0f71f482a...  2022-03   0.969845   
4  62823a01dfebac3445a665d4e0ebc92af01106c8384c26...  2022-03  -0.813313   

    Latitude  LSOA_code                          Type  \
0  50.803304  E01031422  Violence and sexual offences   
1  50.804178  E01031400  Violence and sexual offences   
2  51.137084  E01024001         Theft from the person   
3  51.124173  E01024013  Violence and sexual offences   
4  51.809402  E01017712  Violence and sexual offences   

                                         Outcome     WD24CD  
0                      Status update unavailable  E05009812  
1  Investigation complete; no suspect identified  E05009805  
2  Investigati

In [14]:
import pandas as pd
import sqlite3

# === 1. Load mapping data (MSOA to Ward) ===
mapping_df = pd.read_csv(
    "C:/Users/20232075/Downloads/Middle_Layer_Super_Output_Area_(2021)_to_Ward_to_LAD_(May_2023)_Lookup_in_England_and_Wales.csv"
)

# === 2. Load house prices (Table 1a, from row 6) ===
prices_df = pd.read_excel(
    "C:/Users/20232075/Downloads/HPSSA Dataset 2 - Median price paid by MSOA.xls",
    sheet_name="1a",
    skiprows=5
)

# === 3. Standardize column names for merging ===
prices_df.rename(columns={'MSOA code': 'MSOA21CD'}, inplace=True)

# === 4. Select only price columns and convert to numeric ===
price_cols = [col for col in prices_df.columns if col.startswith("Year ending")]
prices_df[price_cols] = prices_df[price_cols].apply(pd.to_numeric, errors='coerce')

# === 5. Compute average price per MSOA ===
prices_df['Average_Price'] = prices_df[price_cols].mean(axis=1, skipna=True)

# === 6. Merge MSOA prices with ward mapping ===
merged_df = pd.merge(mapping_df, prices_df[['MSOA21CD', 'Average_Price']], on='MSOA21CD', how='inner')

# === 7. Aggregate to average price per ward ===
ward_avg_df = merged_df.groupby('WD23CD')['Average_Price'].mean().reset_index()

# === 8. Save to SQLite ===
conn = sqlite3.connect("crime_data.db")
ward_avg_df.to_sql("ward_average_prices", conn, if_exists="replace", index=False)

# === 9. Verify: Print sample ===
result = pd.read_sql("SELECT * FROM ward_average_prices LIMIT 10", conn)
print(result)

conn.close()


      WD23CD  Average_Price
0  E05000932  159595.261364
1  E05000933  136656.181818
2  E05000934  165505.531818
3  E05000935  124678.690909
4  E05000936   98020.409091
5  E05000937   64977.045455
6  E05000938  141249.540909
7  E05000939   99453.177273
8  E05000940  209185.201515
9  E05000941  120531.763636


C:\Users\20232075\AppData\Local\Temp\ipykernel_67208\1689014390.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  prices_df['Average_Price'] = prices_df[price_cols].mean(axis=1, skipna=True)


In [13]:
import os
os.remove("msoa_ward_prices.db")

In [15]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("crime_data.db")

# Read directly into a pandas DataFrame
df = pd.read_sql_query("SELECT Distinct LSOA_code from crime", conn)

print(df)

conn.close()

       LSOA_code
0      E01031422
1      E01031400
2      E01024001
3      E01024013
4      E01017712
...          ...
10911  E01032014
10912  E01023256
10913  E01033249
10914  E01032413
10915  E01025595

[10916 rows x 1 columns]


In [22]:
import os
import pandas as pd

def get_distinct_column_values(folder_path, column_name):
    distinct_values = set()
    file_count = 0

    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('-street.csv'):
                file_path = os.path.join(root, filename)
                try:
                    df = pd.read_csv(file_path)
                    file_count += 1
                    if column_name in df.columns:
                        values = df[column_name].dropna().unique()
                        distinct_values.update(values)
                    else:
                        print(f"Column '{column_name}' not found in {filename}")
                except Exception as e:
                    print(f"Failed to process {filename}: {e}")

    return distinct_values, file_count

# Example usage
folder_path = "C:/Users/20232075/Desktop/London Police Data"
column_name = "LSOA code"
distinct_values, file_count = get_distinct_column_values(folder_path, column_name)

print(f"Found {len(distinct_values)} distinct values:")
print(distinct_values)
print(f"Processed {file_count} files.")


Found 10918 distinct values:
{'E01018784', 'E01033630', 'E01006273', 'E01000077', 'E01000313', 'E01017886', 'E01000467', 'E01028950', 'E01003368', 'E01002460', 'E01000792', 'E01002573', 'E01024780', 'E01015986', 'E01033602', 'E01003833', 'E01006755', 'E01003764', 'E01018100', 'E01021601', 'E01005835', 'E01002666', 'E01001613', 'E01000460', 'E01004340', 'E01024047', 'E01007041', 'E01001566', 'E01023901', 'E01021550', 'E01001229', 'E01023614', 'E01033437', 'E01017845', 'E01002326', 'E01001809', 'E01004236', 'E01018009', 'E01002767', 'E01003243', 'E01003281', 'E01030482', 'E01003176', 'E01034175', 'E01004075', 'E01032634', 'E01034212', 'E01023814', 'E01002194', 'E01003699', 'E01001940', 'E01011531', 'E01002940', 'E01001583', 'E01011571', 'E01023963', 'E01009202', 'E01031444', 'E01017376', 'E01001931', 'E01002043', 'E01013713', 'E01034177', 'E01007903', 'E01000225', 'E01011992', 'E01002842', 'E01003660', 'E01024162', 'E01028101', 'E01004108', 'E01034184', 'E01000398', 'E01016477', 'E010172

In [5]:
import geopandas as gpd
import pandas as pd
import sqlite3
from shapely.geometry import Point
from tqdm import tqdm

# === 1. Load the full LSOA shapefile ===
lsoa_path = r"C:\Users\20232075\Desktop\cbl\ShapeData\LSOA_2021_EW_BSC_V4.shp"
lsoa_gdf = gpd.read_file(lsoa_path)

# Convert coordinate reference system to WGS84 (lat/lon)
lsoa_gdf = lsoa_gdf.to_crs("EPSG:4326")

# === 2. Load crime data from SQLite ===
db_path = "crime_data.db"
conn = sqlite3.connect(db_path)

# Load data with rowid, lat/lon
crime_df = pd.read_sql_query("SELECT rowid, crimeID, Latitude, Longitude FROM crime", conn)
conn.close()

# Drop rows with missing coordinates
crime_df = crime_df.dropna(subset=["Latitude", "Longitude"])

# === 3. Convert to GeoDataFrame with Points ===
geometry = [Point(xy) for xy in zip(crime_df['Longitude'], crime_df['Latitude'])]
points_gdf = gpd.GeoDataFrame(crime_df, geometry=geometry, crs="EPSG:4326")

# === 4. Perform spatial join to get LSOA21CD ===
joined = gpd.sjoin(points_gdf, lsoa_gdf[['LSOA21CD', 'geometry']], how="left", predicate="within")

# Keep only necessary columns and drop those with no LSOA match
updates = joined[['rowid', 'LSOA21CD']].dropna()

# === 5. Update database with new LSOA_code values ===
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Updating {len(updates)} rows with new LSOA_code values...")

for _, row in tqdm(updates.iterrows(), total=len(updates)):
    cursor.execute("""
        UPDATE crime
        SET LSOA_code = ?
        WHERE rowid = ?
    """, (row['LSOA21CD'], int(row['rowid'])))

conn.commit()
conn.close()

print("LSOA_code column successfully updated based on lat/lon.")


Updating 3258613 rows with new LSOA_code values...


100%|██████████| 3258613/3258613 [01:49<00:00, 29822.43it/s]


LSOA_code column successfully updated based on lat/lon.
